In [50]:
import requests
import json
import pandas as pd 
import numpy as np 
import os
from pprint import pprint
from dotenv import load_dotenv
from sqlalchemy import create_engine
from tqdm import tqdm
from sqlalchemy import Table, Column, MetaData, String, Numeric, JSON, DateTime, Integer, Boolean
from sqlalchemy.dialects.postgresql import JSONB
from datetime import datetime, timedelta

load_dotenv()
partner_token = os.getenv('PARTNER_TOKEN')
login = os.getenv('LOGIN')
password = os.getenv('PASSWORD')
company_id = os.getenv('COMPANY_ID')
partner_id = os.getenv('PARTNER_ID')

print(repr(partner_token))

'cjt8gsr4we7pt2td6xsj'


## Авторизация и получение данных о пользователе

In [2]:
url = 'https://api.yclients.com/api/v1/auth'

headers = {
    "Authorization": f"Bearer {partner_token}",
    "Accept": "application/vnd.yclients.v2+json"
}
payload = {
    "login": login,
    "password": password
}

response = requests.post(url, headers=headers, json=payload)
user_token = response.json()['data']['user_token'] # получаем юзер токен
print(response.status_code)

201


## Информация по компании

In [58]:
url = f"https://api.yclients.com/api/v1/company/{company_id}"

headers = {
    "Accept": "application/vnd.yclients.v2+json",
    "Authorization": partner_token
}

response = requests.get(url, headers=headers)
response.json()['data']

{'id': 1021433,
 'title': 'Monada',
 'public_title': 'Monada',
 'short_descr': 'Салон красоты',
 'logo': 'https://assets.yclients.com/general/b/b4/b49b6c35692b707_20240518125410.png',
 'country_id': 1,
 'country': 'Россия',
 'city_id': 20,
 'city': 'Красноярск',
 'active': 1,
 'phone': '+7 391 200-81-83',
 'phones': ['+7 391 200-81-83'],
 'email': '',
 'timezone': 7,
 'timezone_name': 'Asia/Krasnoyarsk',
 'schedule': '09:00 - 21:00',
 'address': 'проспект Мира, 86',
 'coordinate_lat': 56.011956,
 'coordinate_lon': 92.865093,
 'app_ios': '',
 'app_android': '',
 'phone_confirmation': False,
 'currency_short_title': '₽',
 'reminds_sms_disabled': False,
 'reminds_sms_default': 1,
 'group_priority': 0,
 'bookform_group_priority': 0,
 'description': '<div>MONADA<br /></div><div>Мы создаем красоту:</div><div>- все виды маникюра и педикюра</div><div>- макияж и укладки</div><div>- уход за бровями и ресницами</div><div>- услуги в 4 руки</div><div>- массаж&nbsp;</div><div><br /></div><div>Если В

## Получение данных

In [4]:
url = f"https://api.yclients.com/api/v1/company/{company_id}/analytics/overall/"

headers = {
    "Accept": "application/vnd.yclients.v2+json",
    "Authorization": f"Bearer {partner_token}, User {user_token}"
}

params = {
    "date_from": "2025-01-01",
    "date_to": "2025-04-24",
}

response = requests.get(url, headers=headers, params=params)
pprint(response.json()['data'])

{'client_stats': {'active_count': 599,
                  'lost_count': 419,
                  'lost_percent': 33,
                  'new_count': 340,
                  'new_percent': 57,
                  'return_count': 259,
                  'return_percent': 43,
                  'total_count': 1262},
 'fullness_stats': {'change_percent': 16,
                    'current_percent': 46.9,
                    'previous_percent': 40.2},
 'income_average_services_stats': {'change_percent': -4,
                                   'currency': {'id': 1,
                                                'is_symbol_after_amount': True,
                                                'iso': 'RUB',
                                                'name': 'Russian Ruble',
                                                'symbol': '₽'},
                                   'current_sum': '1983.2',
                                   'previous_sum': '2084.17'},
 'income_average_stats': {'change_percent': 

## Получить данные всех сущностей

### Задаем параметры для API yclients

In [5]:
url = f"https://api.yclients.com/api/v1/company/{company_id}/clients/search"

headers = {
    "Accept": "application/vnd.yclients.v2+json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {partner_token}, User {user_token}"
}

### Задаем параметры для поключения к БД

In [6]:
# === Данные подключения к PostgreSQL ===
db_user = 'user'
db_password = 'password'
db_name = 'Monada'
db_host = 'localhost'
db_port = '5433'

# Строка подключения
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

### Создаем функцию для загрузки всех страниц из API 

In [7]:
# === Функция для загрузки всех страниц из API ===
def get_all_pages(url, method='GET', body=None):
    page = 1
    all_data = []

    with tqdm(desc=f"Загрузка {url.split('/')[-1]}") as pbar:
        while True:
            params_or_json = {"page": page, "page_size": 100}
            if method == 'GET':
                response = requests.get(url, headers=headers, params=params_or_json)
            else:
                if body:
                    body.update(params_or_json)
                response = requests.post(url, headers=headers, json=body)

            if response.status_code != 200:
                print(f"Ошибка запроса: {response.status_code} {response.text}")
                break

            data = response.json().get('data', [])
            if not data:
                break

            all_data.extend(data)
            page += 1
            pbar.update(len(data))

    return all_data

### Создаем функцию для преобразования списков и словарей в JSON строки

In [ ]:
def normalize_json_columns(df):
    """
    Преобразует все столбцы, содержащие списки или словари, в строки JSON.
    """
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, (dict, list))).any():
            df[col] = df[col].apply(lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, (dict, list)) else x)
    return df

### Создаем функции получения данных каждой сущности

In [ ]:

# === Функции для каждой сущности ===

def get_clients():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/clients/search"
    body = {
        "fields": [
            "id",
            "name",
            "surname",
            "phone",
            "email",
            "birthday",
            "card",
            "created_at",
            "last_appointment_date",
            "appointments_count",
            "tags",
            "is_new",
            "custom_fields"
        ]
    }
    df = pd.DataFrame(get_all_pages(url, method='POST', body=body))
    return normalize_json_columns(df)

def get_records():
    url = f"https://api.yclients.com/api/v1/records/{company_id}"
    data = get_all_pages(url, method='GET')
    
    df = pd.json_normalize(data, sep='_')  # Убираем вложенность через нижнее подчеркивание
    return normalize_json_columns(df)

def get_staff():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/staff"
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)

def get_services():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/services"
    body = {
            "fields": [
            "id",
            "company_id",
            "title",
            "description",
            "cost",
            "cost_to_pay",
            "cost_per_unit",
            "amount",
            "discount",
            "first_cost",
            "is_active",
            "category_id"
        ]
    }
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)

def get_goods():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/goods"
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)

def get_goods_transactions():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/goods/transactions"
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)

def get_company():
    url = f"https://api.yclients.com/api/v1/company/{company_id}"
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)

def get_finance_pay_methods():
    url = f"https://api.yclients.com/api/v1/finance/pay_methods/{company_id}"
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)

def get_finance_operations():
    url = f"https://api.yclients.com/api/v1/finance/operations/{company_id}"
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)

def get_schedules():
    url = f"https://api.yclients.com/api/v1/company/{company_id}/schedules"
    df = pd.DataFrame(get_all_pages(url, method='GET'))
    return normalize_json_columns(df)


### Создаем функцию для создания таблиц с правильными типами данных в БД

In [ ]:
# === Функция для создания таблиц с правильными типами данных в БД ===

def create_table_with_types(df, table_name, engine):
    metadata = MetaData()
    columns = []

    for col in df.columns:
        sample_value = df[col].dropna().iloc[0] if not df[col].dropna().empty else None

        if isinstance(sample_value, dict) or isinstance(sample_value, list):
            col_type = JSONB
        elif isinstance(sample_value, bool):
            col_type = Boolean
        elif isinstance(sample_value, int):
            col_type = Integer
        elif isinstance(sample_value, float):
            col_type = Numeric
        elif isinstance(sample_value, datetime.datetime):
            col_type = DateTime
        else:
            col_type = String

        columns.append(Column(col, col_type))

    table = Table(table_name, metadata, *columns)
    metadata.drop_all(engine, [table], checkfirst=True)  # Удалить если уже есть (заменить)
    metadata.create_all(engine)  # Создать таблицу с нужными типами
    print(f"✅ Таблица {table_name} создана с правильными типами колонок.")

### Создаем функцию для заливки датафрейма в БД

In [ ]:
# === Функция для заливки датафрейма в БД ===

def upload_to_postgres(df, table_name):
    if not df.empty:
        create_table_with_types(df, table_name, engine)
        df.to_sql(table_name, engine, if_exists='append', index=False)
        print(f"📥 Данные загружены в таблицу {table_name} ({len(df)} строк).")
    else:
        print(f"⚠️ Таблица {table_name} пуста, пропущена.")

### Получаем данные и загружаем их в БД

In [ ]:
# === Основной процесс ===

print("🚀 Начинаем полную загрузку данных из YClients и запись в PostgreSQL...\n")

entities = {
    "clients": get_clients,
    "records": get_records,
    "staff": get_staff,
    "services": get_services,
    "goods": get_goods,
    "goods_transactions": get_goods_transactions,
    "companies": get_company,
    "finance_operations": get_finance_operations,
    "schedules": get_schedules
}

for table_name, func in entities.items():
    print(f"\n=== {table_name.upper()} ===")
    df = func()
    upload_to_postgres(df, table_name)

print("\n🎉 Все данные успешно загружены в БД!")